In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import shutil, os
import open3d as o3d
from open3d import core as o3c

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Load calib results
proj_mtx =  np.load('../data/proj_mtx.npy', )
proj_dist = np.load('../data/proj_dist.npy', )
cam_mtx =   np.load('../data/cam_mtx.npy', )
cam_dist =  np.load('../data/cam_dist.npy')
proj_R =    np.load('../data/R.npy')
proj_T =    np.load('../data/T.npy')
R1 =        np.load('../data/R1.npy')
R2 =        np.load('../data/R2.npy')
P1 =        np.load('../data/P1.npy')
P2 =        np.load('../data/P2.npy')

proj_w, proj_h = (1920, 1080)
cam_w, cam_h = (1920, 1080)
scale_x = 1920 / proj_w
scale_y = 1080 / proj_h
proj_mtx[0, :] = proj_mtx[0, :] * scale_x
proj_mtx[1, :] = proj_mtx[1, :] * scale_y

base_path = '../data/recordings/record_0/'
h_pixels = np.load(base_path+'h_pixels.npy')
v_pixels = np.load(base_path+'v_pixels.npy')

In [9]:
cam_pts = []
proj_pts = []
colors = []
# Extract color
img_white = cv2.imread(base_path + '/frame_1.jpg', cv2.IMREAD_COLOR)
img_white = cv2.cvtColor(img_white, cv2.COLOR_BGR2RGB)

for i in range(500, 1300):
    for j in range(200, 800):
# for i in range(1920):
#     for j in range(1080):
        h_value = h_pixels[j, i]
        v_value = v_pixels[j, i]
        if h_value == -1 or v_value == -1:
            pass
        else:
            cam_pts.append([i,j])
            h_value = min(proj_w-1, h_value)
            v_value = min(proj_h-1, v_value)
            proj_pts.append([ h_value, v_value])
            colors.append(img_white[j, i, :])

cam_pts = np.array(cam_pts, dtype=np.float32)
proj_pts = np.array(proj_pts, dtype=np.float32)

In [15]:
# newcameramtx, roi = cv2.getOptimalNewCameraMatrix(cam_mtx, cam_dist, (800,600), 1, (800,600))
# newprojmtx, roi = cv2.getOptimalNewCameraMatrix(proj_mtx, proj_dist, (1920,1080), 1, (1920,1080))
# cam_pts_homo = cv2.undistortPoints(np.expand_dims(cam_pts, axis=1), cam_mtx, cam_dist, None, newcameramtx)
# proj_pts_homo = cv2.undistortPoints(np.expand_dims(proj_pts, axis=1), proj_mtx, proj_dist, None, newprojmtx)

cam_pts_homo = cv2.convertPointsToHomogeneous(cv2.undistortPoints(np.expand_dims(cam_pts, axis=1), cam_mtx, cam_dist, R=proj_R))[:,0].T
proj_pts_homo = cv2.convertPointsToHomogeneous(cv2.undistortPoints(np.expand_dims(proj_pts, axis=1), proj_mtx, proj_dist))[:,0].T
T = proj_T[:,0]

# Took from https://github.com/caoandong/Projector_Calibration/blob/master/visual_calib.py
TLen = np.linalg.norm(T)
NormedL = cam_pts_homo/np.linalg.norm(cam_pts_homo, axis=0)
alpha = np.arccos(np.dot(-T, NormedL)/TLen)
degalpha = alpha*180/np.pi
beta = np.arccos(np.dot(T, proj_pts_homo)/(TLen*np.linalg.norm(proj_pts_homo, axis=0)))
degbeta = beta*180/np.pi
gamma = np.pi - alpha - beta
P_len = TLen*np.sin(beta)/np.sin(gamma)
Pts = NormedL*P_len


colors_array = np.array(colors).astype(np.float64)/255.0

pcd = o3d.t.geometry.PointCloud(o3c.Tensor(Pts.T, o3c.float32))
pcd = pcd.to_legacy()
pcd.colors = o3d.cpu.pybind.utility.Vector3dVector(colors_array)

o3d.visualization.draw_geometries([pcd])